## Requirements

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install opendatasets
!pip install pydub
!pip install gdown

In [3]:
import torch
from transformers import AutoProcessor, Wav2Vec2Model
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from time import time
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import os
from pydub import AudioSegment
import opendatasets as od
import pandas as pd
import gc
import random
import pickle
import gdown

## Load Files

In [4]:
url = "https://drive.google.com/file/d/1klayW8tzpkmhS6n6Zb3ljvDG40SakMjR/view?usp=sharing"
output = "audios_dataset.npy"
gdown.download(url, output, quiet=False, fuzzy=True)

url = "https://drive.google.com/file/d/1-1HvdclgtF-geqbVJC8sOxJ0Q2oLS6ex/view?usp=share_link"
output = "keywords_dataset.pkl"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1klayW8tzpkmhS6n6Zb3ljvDG40SakMjR
To: /kaggle/working/audios_dataset.npy
100%|██████████| 4.79G/4.79G [00:47<00:00, 102MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1-1HvdclgtF-geqbVJC8sOxJ0Q2oLS6ex
To: /kaggle/working/keywords_dataset.pkl
100%|██████████| 109k/109k [00:00<00:00, 59.7MB/s]


'keywords_dataset.pkl'

In [6]:
# Loading data
with open('keywords_dataset.pkl', 'rb') as f:
    keywords_dataset = pickle.load(f)
audios_dataset = np.load('audios_dataset.npy', allow_pickle=True)
print(len(keywords_dataset), len(audios_dataset))

3000 3000


## Load Models

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
from datasets import load_dataset
from transformers import AutoProcessor, ClapModel

# dataset = load_dataset("ashraq/esc50")
# audio_sample = dataset["train"]["audio"][0]["array"]

model = ClapModel.from_pretrained("laion/clap-htsat-unfused").to(device)
processor = AutoProcessor.from_pretrained("laion/clap-htsat-unfused")



In [10]:
def text_audio_similarity(input_text, audio_sample, model=model, processor=processor):
    inputs = processor(text=input_text, audios=audio_sample, return_tensors="pt", padding=True).to(device)
    outputs = model(**inputs)
    logits_per_audio = outputs.logits_per_text
    probs = logits_per_audio.softmax(dim=-1)
    return probs